In [2]:
import pandas as pd
import os
import glob
from sklearn import preprocessing
from collections import deque
import numpy as np
import random
from multiprocessing import pool
import pickle
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import datetime, os
import random

In [2]:
dirac = 'D:\ML\crypto_data\crypto_data'
name = ['Time','Low','High','Open','Close','Volume']
txt = ['BCH-USD','BTC-USD','ETH-USD','LTC-USD']

In [22]:
seq_len = 60
future_pred_len = 3
ratio_to_pred = 'LTC-USD'
Epochs = 10
batch_size = 64
Name = f'{seq_len}-seq-{future_pred_len}-pred-{int(time.time())}' 

In [5]:
def classify(current,future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [6]:
def preprocess_df(df):
    df = df.drop('future',1)
    for col in df.columns:
        if col != 'target':
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col])
            df.dropna(inplace=True)
    seq_data = []
    prev_day = deque(maxlen=seq_len)
    for i in df.values:
        prev_day.append([n for n in i[:-1]])
        if len(prev_day) == seq_len:
            seq_data.append([np.array(prev_day),i[-1]])
            np.random.shuffle(seq_data)
    buys = []
    sells = []
    
    for seq,target in seq_data:
        if target == 1:
            buys.append([seq,target])
        elif target == 0:
            sells.append([seq,target])
            
    
    
    lower = min(len(buys),len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    seq_data = buys+sells
    np.random.shuffle(seq_data)
    
    X = []
    y = []
    
    for seq , target in seq_data:
        X.append(seq)
        y.append(target)
        
    return np.array(X),np.array(y)

In [7]:
final_data = pd.DataFrame()
for i in txt:
    df = pd.read_csv(f'{dirac}\{i}.csv',names=name)
    df.rename(columns={'Close':f'{i}_close','Volume':f'{i}_vol'},inplace=True)
    df.set_index('Time',inplace=True)
    df = df[[f'{i}_close',f'{i}_vol']]
    if len(final_data)==0:
        final_data=df
    else:
        final_data = final_data.join(df)

In [8]:
final_data

,BCH-USD_close,BCH-USD_vol,BTC-USD_close,BTC-USD_vol,ETH-USD_close,ETH-USD_vol,LTC-USD_close,LTC-USD_vol
Time,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200
1528968720,870.859985,26.856577,6487.379883,7.706374,486.010010,26.019083,96.660004,314.387024
1528968780,870.099976,1.124300,6479.410156,3.088252,486.000000,8.449400,96.570000,77.129799
1528968840,870.789978,1.749862,6479.410156,1.404100,485.750000,26.994646,96.500000,7.216067
1528968900,870.000000,1.680500,6479.979980,0.753000,486.000000,77.355759,96.389999,524.539978
...,...,...,...,...,...,...,...,...
1535215020,531.479980,0.016868,6714.520020,1.002652,279.359985,8.790519,58.009998,7.301921
1535215080,531.469971,0.013854,6714.520020,1.021925,279.369995,1.311763,58.020000,23.802017
1535215140,531.479980,0.016900,6715.000000,3.645508,279.660004,11.752819,58.020000,6.953497


In [9]:
final_data['future'] = final_data[f'{ratio_to_pred}_close'].shift(-future_pred_len) 

In [10]:
final_data['target']  =list(map(classify,final_data[f'{ratio_to_pred}_close'],final_data['future']))

In [11]:
final_data

,BCH-USD_close,BCH-USD_vol,BTC-USD_close,BTC-USD_vol,ETH-USD_close,ETH-USD_vol,LTC-USD_close,LTC-USD_vol,future,target
Time,,,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200,96.500000,0
1528968720,870.859985,26.856577,6487.379883,7.706374,486.010010,26.019083,96.660004,314.387024,96.389999,0
1528968780,870.099976,1.124300,6479.410156,3.088252,486.000000,8.449400,96.570000,77.129799,96.519997,0
1528968840,870.789978,1.749862,6479.410156,1.404100,485.750000,26.994646,96.500000,7.216067,96.440002,0
1528968900,870.000000,1.680500,6479.979980,0.753000,486.000000,77.355759,96.389999,524.539978,96.470001,1
...,...,...,...,...,...,...,...,...,...,...
1535215020,531.479980,0.016868,6714.520020,1.002652,279.359985,8.790519,58.009998,7.301921,58.080002,1
1535215080,531.469971,0.013854,6714.520020,1.021925,279.369995,1.311763,58.020000,23.802017,58.090000,1
1535215140,531.479980,0.016900,6715.000000,3.645508,279.660004,11.752819,58.020000,6.953497,NaN,0


In [12]:
times = sorted(final_data.index.values)

In [13]:
last_5per = times[-int(0.05*len(times))]
last_5per

1534879920

In [14]:
val_df = final_data[(final_data.index>=last_5per)]
train_df = final_data[(final_data.index<=last_5per)] 

In [15]:
val_X,val_y = preprocess_df(val_df)

In [16]:
train_X,train_y = preprocess_df(train_df)

KeyboardInterrupt: 

In [ ]:
out = open(r'C:\Users\Sampath HN\crypto\train_x.pickle','wb')
pickle.dump(train_X,out)
out.close

out = open(r'C:\Users\Sampath HN\crypto\train_y.pickle','wb')
pickle.dump(train_y,out)
out.close

out = open(r'C:\Users\Sampath HN\crypto\val_X.pickle','wb')
pickle.dump(val_X,out)
out.close

out = open(r'C:\Users\Sampath HN\crypto\val_y.pickle','wb')
pickle.dump(val_y,out)
out.close

In [3]:
train_X = pickle.load(open('train_x.pickle','rb'))
train_y = pickle.load(open('train_y.pickle','rb'))
val_X = pickle.load(open('val_x.pickle','rb'))
val_y = pickle.load(open('val_y.pickle','rb'))

In [4]:
model = Sequential()

In [5]:
model.add(LSTM(128,input_shape =train_X.shape[1:],return_sequences=True))

In [6]:
model.add(Dropout(0.2))

In [7]:
model.add(BatchNormalization())

In [8]:
model.add(LSTM(128,input_shape =(60,8),return_sequences=True))

In [9]:
model.add(Dropout(0.2))

In [10]:
model.add(BatchNormalization())

In [11]:
model.add(LSTM(128,input_shape =(60,8)))

In [12]:
model.add(Dropout(0.2))

In [13]:
model.add(BatchNormalization())

In [14]:
model.add(Dense(32,activation='relu'))

In [15]:
model.add(Dropout(0.2))

In [16]:
model.add(Dense(2,activation='softmax'))

In [17]:
opt = tf.keras.optimizers.Adam(lr=0.001,decay=1e-6)

In [18]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [19]:
%load_ext tensorboard

In [20]:
log_dir = os.path.join('logs','fit','') + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
history = model.fit(train_X,train_y,batch_size=batch_size,epochs=Epochs,validation_data=(val_X,val_y),callbacks = [tensorboard_callback])

Train on 69402 samples, validate on 3688 samples
Epoch 1/10
69402/69402 [==============================] - 27s 383us/sample - loss: 0.6867 - accuracy: 0.5490 - val_loss: 0.6806 - val_accuracy: 0.5624
Epoch 2/10
69402/69402 [==============================] - 23s 328us/sample - loss: 0.6836 - accuracy: 0.5570 - val_loss: 0.6761 - val_accuracy: 0.5713
Epoch 3/10
69402/69402 [==============================] - 23s 331us/sample - loss: 0.6814 - accuracy: 0.5637 - val_loss: 0.6807 - val_accuracy: 0.5691
Epoch 4/10
69402/69402 [==============================] - 21s 298us/sample - loss: 0.6799 - accuracy: 0.5676 - val_loss: 0.6792 - val_accuracy: 0.5607
Epoch 5/10
69402/69402 [==============================] - 23s 325us/sample - loss: 0.6779 - accuracy: 0.5736 - val_loss: 0.6809 - val_accuracy: 0.5727
Epoch 6/10
69402/69402 [==============================] - 23s 329us/sample - loss: 0.6761 - accuracy: 0.5799 - val_loss: 0.6796 - val_accuracy: 0.5746
Epoch 7/10
69402/69402 [=====================

In [56]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 60, 128)           70144     
_________________________________________________________________
dropout_7 (Dropout)          (None, 60, 128)           0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 60, 128)           512       
_________________________________________________________________
lstm_7 (LSTM)                (None, 60, 128)           131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 60, 128)           0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 60, 128)           512       
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)              

In [25]:
%tensorboard --logdir logs/gradient_tape

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9444.

In [ ]:
model.save('')